In [1]:
!pip install skorch pytorch_lightning einops torcheval

In [2]:
import torch
import torch.nn.functional as F

F.one_hot(torch.Tensor([5, 2]).to(torch.long), num_classes=10)

tensor([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms
from torchvision.datasets import MNIST


def create_dataloader(batch_size):
    # target_transform = transforms.Compose([
    #     lambda x: print("hoge"),
    #     lambda x: print(x),
    #     torch.tensor,
    #     lambda x: F.one_hot(x, num_classes=10),
    # ])
    
    data_train = torch.utils.data.DataLoader(
        MNIST(
            '~/mnist_data', train=True, download=True,
            transform=transforms.Compose([
                transforms.ToTensor
            ]),
            # target_transform=target_transform,
        ),
        batch_size=batch_size,
        shuffle=True
    )

    data_test = torch.utils.data.DataLoader(
        MNIST(
            '~/mnist_data', train=False, download=True,
            transform=transforms.Compose([
                transforms.ToTensor
            ]),
            # target_transform=target_transform,
        ),
        batch_size=batch_size,
        shuffle=True
    )

    return data_train, data_test


In [4]:
batch_size = 64

trainloader, testloader = create_dataloader(batch_size=batch_size)

Failed to download (trying next):
<urlopen error [Errno 8] nodename nor servname provided, or not known>



100%|██████████| 9912422/9912422 [00:04<00:00, 2025402.24it/s]


Extracting /Users/jimako/mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to /Users/jimako/mnist_data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 8] nodename nor servname provided, or not known>



100%|██████████| 28881/28881 [00:00<00:00, 157476.97it/s]


Extracting /Users/jimako/mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/jimako/mnist_data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 8] nodename nor servname provided, or not known>



100%|██████████| 1648877/1648877 [00:01<00:00, 1610805.37it/s]


Extracting /Users/jimako/mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/jimako/mnist_data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 8] nodename nor servname provided, or not known>



100%|██████████| 4542/4542 [00:00<00:00, 2596855.07it/s]

Extracting /Users/jimako/mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/jimako/mnist_data/MNIST/raw



In [5]:
from s4.skorch import S4Classifier

model = S4Classifier(
    max_epochs=10,
    optimizer=torch.optim.Adam,
    lr=0.001,
    batch_size=batch_size,
    criterion=nn.CrossEntropyLoss,
    d_input=784,
    d_output=10,
    d_model=512,
    n_layers=3,
    dropout=[0.2]*3,
    transposed=False,
    s4d=True,
)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CUDA extension for structured kernels (Cauchy and Vandermonde multiplication) not found. Install by going to extensions/kernels/ and running `python setup.py install`, for improved speed and memory efficiency. Note that the kernel changed for state-spaces 4.0 and must be recompiled.
Falling back on slow Cauchy and Vandermonde kernel. Install at least one of pykeops or the CUDA extension for better speed and memory efficiency.


No module named 'extensions'


In [6]:
model

<class 's4.skorch.S4Classifier'>[uninitialized](
  module=SingleS4Classifier(
    (encoder): Sequential(
      (0): Linear(in_features=784, out_features=512, bias=True)
      (1): GELU(approximate='none')
    )
    (decoder): Sequential(
      (0): Linear(in_features=512, out_features=10, bias=True)
      (1): LogSoftmax(dim=1)
    )
  ),
)

In [5]:
import numpy as np

X_train = trainloader.dataset.data.view(-1, 784).numpy().astype(np.float32)
y_train = F.one_hot(trainloader.dataset.targets, num_classes=10).view(-1, 10).numpy().astype(np.float32)

X_test = testloader.dataset.data.view(-1, 784).numpy().astype(np.float32)
y_test = F.one_hot(testloader.dataset.targets, num_classes=10).view(-1, 10).numpy().astype(np.float32)

In [8]:
model.fit(X_train, y_train)

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        1.6470        1.1495  47.5857
      2        1.1162        1.0746  48.1060
      3        1.0811        1.0703  46.6298
      4        1.0542        1.0533  47.9270
      5        1.0386        1.0815  46.0384
      6        1.0291        1.0604  45.8468
      7        1.0266        1.0396  44.7378
      8        1.0106        1.0246  45.1809
      9        1.0084        1.0480  45.6608
     10        0.9982        1.0692  45.0144


<class 's4.skorch.S4Classifier'>[initialized](
  module_=SingleS4Classifier(
    (encoder): Sequential(
      (0): Linear(in_features=784, out_features=512, bias=True)
      (1): GELU(approximate='none')
    )
    (decoder): Sequential(
      (0): Linear(in_features=512, out_features=10, bias=True)
      (1): LogSoftmax(dim=1)
    )
  ),
)

In [9]:
pred = model.predict(X_test)

In [10]:
from pprint import pprint

from torcheval.metrics.functional import multiclass_accuracy
from torcheval.metrics.functional import multiclass_precision
from torcheval.metrics.functional import multiclass_recall
from torcheval.metrics.functional import multiclass_f1_score

p = {
    "input": torch.argmax(torch.Tensor(pred), dim=1),
    "target": torch.argmax(torch.Tensor(y_test), dim=1),
    "num_classes": 10
}

pprint({
    "accuracy": multiclass_accuracy(**p).item(),
    "precision": multiclass_precision(**p).item(),
    "recall": multiclass_recall(**p).item(),
    "f1_score": multiclass_f1_score(**p).item(),
})

{'accuracy': 0.6733999848365784,
 'f1_score': 0.6733999848365784,
 'precision': 0.6733999848365784,
 'recall': 0.6733999848365784}
